In [1]:
%matplotlib inline
%load_ext autoreload

%autoreload 2

# pandas 모듈은 흔히 pd라는 약칭으로 사용된다. Global 표준에 가깝다.
import pandas as pd
import time
import json
from IPython.display import HTML
from IPython.display import display

# 커스텀 라이브러리를 import한다.
from pfx_plot import *
from misc import *

# precision 세팅을 한다. 내부 값은 소수점 5자리까지 표시되도록 저장하고, display 시에는 1자리로 표시하도록 한다.
pd.set_option('precision', 5)
pd.set_option('display.float_format', lambda x: '%.1f' % x)

set_fonts()

`JSON` 파일은 모바일 페이지에서 긁어온 정보를 취사선택, 취합한 것임.

In [82]:
fname = 'pbp_data/2019/10/20191001NCOB02019_relay.json'
f = open(fname, 'r')
js = json.load(f)
f.close()
rl = js.get('relayList')

stadium = js.get('stadium')
referee = js.get('referee')

In [83]:
tl_keys = []
rl_keys = []
pts_keys = []
for k in rl.keys():
    keys = rl.get(k).keys()
    for key in keys:
        if key in rl_keys:
            continue
        else:
            rl_keys.append(key)
    
    for j in range(len(rl.get(k).get('textOptionList'))):
        keys = rl.get(k).get('textOptionList')[j].keys()
        for key in keys:
            if key in tl_keys:
                continue
            else:
                tl_keys.append(key)
    for j in range(len(rl.get(k).get('ptsOptionList'))):
        keys = rl.get(k).get('ptsOptionList')[j].keys()
        for key in keys:
            if key in pts_keys:
                continue
            else:
                pts_keys.append(key)
# rl_keys, pts_keys, tl_keys

In [84]:
tl_keys_copy = tl_keys.copy()
tl_keys_copy.remove('currentGameState')
tl_keys_copy.remove('batterRecord')
tl_keys_copy.remove('playerChange')
tl_keys_copy.remove('pitcherResult')

In [85]:
ts_set = []
for k in rl.keys():
    for j in range(len(rl.get(k).get('textOptionList'))):
        ts = rl.get(k).get('textOptionList')[j]
        
        ts_dict = {}
        ts_dict['textOrder'] = int(k)
        for key in tl_keys_copy:
            ts_dict[key] = None if key not in ts.keys() else ts.get(key)
        ts_dict['referee'] = referee
        ts_dict['stadium'] = stadium
        ts_set.append(ts_dict)
ts_df = pd.DataFrame(ts_set)
ts_df = ts_df.rename(index=str, columns={'ptsPitchId': 'pitchId'})
# ts_df.head()

In [86]:
pdata_set = []
if len(pts_keys) > 0:
    for k in rl.keys():
        for j in range(len(rl.get(k).get('ptsOptionList'))):
            pdata = rl.get(k).get('ptsOptionList')[j]

            pdata_dict = {}
            pdata_dict['textOrder'] = int(k)
            for key in pts_keys:
                pdata_dict[key] = None if key not in pdata.keys() else pdata.get(key)
            pdata_set.append(pdata_dict)
        
    pdata_df = pd.DataFrame(pdata_set)
    pdata_df.head()
else:
    pdata_df = None

if pdata_df is not None:
    merge_df = pd.merge(ts_df, pdata_df, how='outer')
    # merge_df.to_csv('test.csv', index=False)
else:
    merge_df = ts_df

# away team lineup

In [87]:
atl = js.get('awayTeamLineUp')
abat = atl.get('batter')
apit = atl.get('pitcher')
bats = pd.DataFrame(abat).sort_values
pits = pd.DataFrame(apit)

In [92]:
bats.sort_values(['posName', 'seqno'])

,bb,ab,posName,seqno,batOrder,seasonHra,birth,weight,cin,run,...,hitType,pos,pCode,hbp,name,rbi,todayHra,so,height,cout
11,1,4,1루수,1,6,0.3,19850508,89.0,NaN,1,...,우투우타,3,78813,0,모창민,0,0.5,1,188.0,true
12,0,1,1루수,4,7,0.3,19890205,82.0,true,1,...,우투우타,3,60566,0,이상호,0,0.0,0,180.0,NaN
5,0,4,2루수,1,3,0.3,19930206,80.0,NaN,0,...,우투좌타,4,62907,0,박민우,1,0.8,0,185.0,true
4,0,1,2루수,2,3,0.2,19840317,74.0,true,0,...,우투우타,4,73306,0,지석훈,0,1.0,0,181.0,NaN
13,1,3,3루수,1,7,0.3,19850622,88.0,NaN,0,...,우투우타,5,74465,0,박석민,0,0.3,0,178.0,true
10,0,0,3루수,3,6,0.3,19971229,80.0,true,0,...,우투우타,5,66917,0,김찬형,0,0.0,0,182.0,NaN
7,0,0,대주자,2,4,0.2,19991102,98.0,NaN,0,...,우투우타,11,68912,0,김형준,0,0.0,0,187.0,true
9,0,5,우익수,1,5,0.2,19890209,95.0,NaN,0,...,우투우타,9,69901,0,스몰린스키,0,0.0,2,180.0,NaN
15,0,4,유격수,1,8,0.3,19890715,80.0,NaN,0,...,우투좌타,6,62931,0,노진혁,0,0.5,0,184.0,true
2,0,0,유격수,2,2,0.3,19801019,73.0,true,0,...,우투우타,6,73213,0,손시헌,0,0.0,0,172.0,NaN


In [88]:
bats

,bb,ab,posName,seqno,batOrder,seasonHra,birth,weight,cin,run,...,hitType,pos,pCode,hbp,name,rbi,todayHra,so,height,cout
0,0,1,지명타자,3,1,0.3,19901230,85.0,true,1,...,우투우타,0,63963,0,권희동,1,1.0,0,177.0,NaN
1,0,4,지명타자,1,1,0.3,19951007,75.0,NaN,1,...,우투좌타,0,64984,0,김태진,0,0.2,2,170.0,true
2,0,0,유격수,2,2,0.3,19801019,73.0,true,0,...,우투우타,6,73213,0,손시헌,0,0.0,0,172.0,NaN
3,0,5,좌익수,1,2,0.3,19871226,87.0,NaN,0,...,좌투좌타,7,76849,0,이명기,0,0.2,1,183.0,true
4,0,1,2루수,2,3,0.2,19840317,74.0,true,0,...,우투우타,4,73306,0,지석훈,0,1.0,0,181.0,NaN
5,0,4,2루수,1,3,0.3,19930206,80.0,NaN,0,...,우투좌타,4,62907,0,박민우,1,0.8,0,185.0,true
6,0,0,포수,3,4,0.2,19891230,95.0,true,0,...,우투우타,2,78122,0,김태군,0,0.0,0,182.0,NaN
7,0,0,대주자,2,4,0.2,19991102,98.0,NaN,0,...,우투우타,11,68912,0,김형준,0,0.0,0,187.0,true
8,0,5,포수,1,4,0.4,19870605,85.0,NaN,0,...,우투우타,2,76232,0,양의지,1,0.4,0,179.0,true
9,0,5,우익수,1,5,0.2,19890209,95.0,NaN,0,...,우투우타,9,69901,0,스몰린스키,0,0.0,2,180.0,NaN


In [89]:
pits

,bb,kk,vsEra,seqno,ballCount,inn,birth,weight,run,hr,...,psEra,backnum,hit,hitType,pCode,hbp,name,wp,todayEra,height
0,1,2,0.00,1,46,2.1,19970428,85.0,0,0,...,0.0,26,3,좌투좌타,66920,0,최성영,0,0.0,180.0
1,0,2,0.00,2,16,1.0,19900710,89.0,0,0,...,0.0,56,0,좌투좌타,79358,0,강윤구,0,0.0,183.0
2,0,0,0.00,3,3,0.2,19850307,92.0,0,0,...,0.0,55,0,우투우타,75867,0,김진성,0,0.0,186.0
3,0,0,0.00,4,19,1.0,19900212,83.0,1,0,...,0.0,30,4,우언우타,63991,0,박진우,0,9.0,180.0
4,0,1,0.00,5,12,1.0,19850825,85.0,0,0,...,0.0,45,0,우투우타,78352,0,임창민,0,0.0,183.0
5,0,0,0.00,6,8,0.1,19911002,85.0,1,0,...,0.0,24,2,우투우타,60339,0,김건태,0,0.0,185.0
6,0,1,0.00,7,18,1.0,19900416,90.0,0,0,...,0.0,65,0,좌투좌타,63959,0,임정호,0,0.0,188.0
7,0,0,0.00,8,18,0.2,19950224,91.0,3,0,...,0.0,28,4,우투우타,63950,0,장현식,0,40.5,181.0
8,0,0,0.00,9,10,0.1,19870731,83.0,1,0,...,0.0,46,2,우언우타,76118,0,원종현,0,27.0,182.0


# pre-2016

In [59]:
fname = 'pbp_data/2008/3/20080329HTSS0_relay.json'
f = open(fname, 'r')
js = json.load(f)
f.close()
rl = js.get('relayList')

stadium = js.get('stadium')
referee = js.get('referee')

In [60]:
tl_keys = []
rl_keys = []
pts_keys = []
for k in rl.keys():
    keys = rl.get(k).keys()
    for key in keys:
        if key in rl_keys:
            continue
        else:
            rl_keys.append(key)
    
    for j in range(len(rl.get(k).get('textOptionList'))):
        keys = rl.get(k).get('textOptionList')[j].keys()
        for key in keys:
            if key in tl_keys:
                continue
            else:
                tl_keys.append(key)
    for j in range(len(rl.get(k).get('ptsOptionList'))):
        keys = rl.get(k).get('ptsOptionList')[j].keys()
        for key in keys:
            if key in pts_keys:
                continue
            else:
                pts_keys.append(key)
# rl_keys, pts_keys, tl_keys

In [61]:
tl_keys_copy = tl_keys.copy()
tl_keys_copy.remove('currentGameState')
tl_keys_copy.remove('batterRecord')
tl_keys_copy.remove('playerChange')
tl_keys_copy.remove('pitcherResult')

In [63]:
ts_set = []
for k in rl.keys():
    for j in range(len(rl.get(k).get('textOptionList'))):
        ts = rl.get(k).get('textOptionList')[j]
        
        ts_dict = {}
        ts_dict['textOrder'] = int(k)
        for key in tl_keys_copy:
            ts_dict[key] = None if key not in ts.keys() else ts.get(key)
        ts_dict['referee'] = referee
        ts_dict['stadium'] = stadium
        ts_set.append(ts_dict)
ts_df = pd.DataFrame(ts_set)
ts_df = ts_df.rename(index=str, columns={'ptsPitchId': 'pitchId'})
ts_df.head()

,textOrder,seqno,text,type,stuff,pitchNum,pitchResult,pitchId,speed,referee,stadium
0,13,74,7번타자 조동찬,8,,nan,None,None,None,김병주,대구
1,13,75,1구 타격,1,체인지업,1.0,H,-1,117,김병주,대구
2,13,76,조동찬 : 3루수 앞 땅볼로 출루,13,,nan,None,None,None,김병주,대구
3,13,77,1루주자 박진만 : 포스아웃 (3루수->2루수 2루 터치아웃),14,,nan,None,None,None,김병주,대구
4,12,65,6번타자 박진만,8,,nan,None,None,None,김병주,대구


In [72]:
pdata_set = []
if len(pts_keys) > 0:
    for k in rl.keys():
        for j in range(len(rl.get(k).get('ptsOptionList'))):
            pdata = rl.get(k).get('ptsOptionList')[j]

            pdata_dict = {}
            pdata_dict['textOrder'] = int(k)
            for key in pts_keys:
                pdata_dict[key] = None if key not in pdata.keys() else pdata.get(key)
            pdata_set.append(pdata_dict)
        
    pdata_df = pd.DataFrame(pdata_set)
    pdata_df.head()
else:
    pdata_df = None

if pdata_df is not None:
    merge_df = pd.merge(ts_df, pdata_df, how='outer')
    # merge_df.to_csv('test.csv', index=False)
else:
    merge_df = ts_df

# away team lineup

In [77]:
atl = js.get('awayTeamLineUp')
abat = atl.get('batter')
apit = atl.get('pitcher')

In [78]:
bats = pd.DataFrame(abat)
pits = pd.DataFrame(apit)

In [80]:
bats

,bb,ab,posName,seqno,batOrder,seasonHra,birth,weight,run,hr,...,pos,pCode,hbp,name,rbi,todayHra,so,height,cin,cout
0,0,5,중견수,1,1,0.6,19850826,70.0,1,0,...,8,74163,0,이용규,0,0.6,0,175.0,NaN,NaN
1,2,3,좌익수,1,2,0.3,19781218,77.0,1,0,...,7,71207,0,김원섭,0,0.3,1,180.0,NaN,NaN
2,3,2,1루수,1,3,0.0,19771018,85.0,0,0,...,3,96610,0,장성호,0,0.0,1,184.0,NaN,NaN
3,0,3,우익수,1,4,0.0,19850519,95.0,0,0,...,9,78629,0,나지완,1,0.0,1,182.0,NaN,NaN
4,0,4,지명타자,1,5,0.0,19790316,109.0,0,0,...,0,77623,0,최희섭,0,0.0,2,196.0,NaN,NaN
5,1,3,3루수,1,6,0.0,19800221,85.0,0,0,...,5,72606,0,이현곤,0,0.0,0,183.0,NaN,NaN
6,0,4,유격수,1,7,0.0,19780520,78.0,0,0,...,6,78649,0,발데스,0,0.0,2,180.0,NaN,NaN
7,0,4,포수,1,8,0.2,19771027,84.0,1,1,...,2,70612,0,김상훈,1,0.2,0,179.0,NaN,NaN
8,0,1,대타,2,9,0.0,19721018,96.0,0,0,...,10,95114,0,심재학,0,0.0,0,184.0,true,NaN
9,0,3,2루수,1,9,0.3,19730914,77.0,0,0,...,4,96616,0,김종국,0,0.3,0,180.0,NaN,true


In [81]:
pits

,bb,kk,vsEra,seqno,ballCount,inn,birth,weight,run,hr,...,psEra,backnum,hit,hitType,pCode,hbp,name,wp,todayEra,height
0,2,1,0.00,1,87,5.1,19720930,95.0,3,0,...,0.0,27,8,우투우타,78627,0,리마,0,5.1,188.0
1,0,2,0.00,2,20,2.0,19770212,83.0,1,0,...,0.0,39,3,우언우타,99639,0,유동훈,0,4.5,183.0
2,0,0,0.00,3,1,0.0,19880301,78.0,0,0,...,0.0,37,1,좌투좌타,77637,0,양현종,0,0.0,183.0
3,1,0,0.00,4,14,0.2,19841009,98.0,0,0,...,0.0,35,0,우투우타,73632,0,임준혁,0,0.0,182.0


# 버그 있던거

In [38]:
fname = 'pbp_data/2019/4/20190423HTLG02019_relay.json'
f = open(fname, 'r')
js = json.load(f)
f.close()
rl = js.get('relayList')

stadium = js.get('stadium')
referee = js.get('referee')

In [39]:
tl_keys = []
rl_keys = []
pts_keys = []
for k in rl.keys():
    keys = rl.get(k).keys()
    for key in keys:
        if key in rl_keys:
            continue
        else:
            rl_keys.append(key)
    
    for j in range(len(rl.get(k).get('textOptionList'))):
        keys = rl.get(k).get('textOptionList')[j].keys()
        for key in keys:
            if key in tl_keys:
                continue
            else:
                tl_keys.append(key)
    for j in range(len(rl.get(k).get('ptsOptionList'))):
        keys = rl.get(k).get('ptsOptionList')[j].keys()
        for key in keys:
            if key in pts_keys:
                continue
            else:
                pts_keys.append(key)
# rl_keys, pts_keys, tl_keys

In [40]:
tl_keys_copy = tl_keys.copy()
tl_keys_copy.remove('currentGameState')
tl_keys_copy.remove('batterRecord')
tl_keys_copy.remove('playerChange')
tl_keys_copy.remove('pitcherResult')

In [41]:
ts_set = []
for k in rl.keys():
    for j in range(len(rl.get(k).get('textOptionList'))):
        ts = rl.get(k).get('textOptionList')[j]
        
        ts_dict = {}
        ts_dict['textOrder'] = int(k)
        for key in tl_keys_copy:
            ts_dict[key] = None if key not in ts.keys() else ts.get(key)
        ts_dict['referee'] = referee
        ts_dict['stadium'] = stadium
        ts_set.append(ts_dict)
ts_df = pd.DataFrame(ts_set)
ts_df = ts_df.rename(index=str, columns={'ptsPitchId': 'pitchId'})
# ts_df.head()

In [42]:
pdata_set = []
if len(pts_keys) > 0:
    for k in rl.keys():
        for j in range(len(rl.get(k).get('ptsOptionList'))):
            pdata = rl.get(k).get('ptsOptionList')[j]

            pdata_dict = {}
            pdata_dict['textOrder'] = int(k)
            for key in pts_keys:
                pdata_dict[key] = None if key not in pdata.keys() else pdata.get(key)
            pdata_set.append(pdata_dict)
        
    pdata_df = pd.DataFrame(pdata_set)
    pdata_df.head()
else:
    pdata_df = None

if pdata_df is not None:
    merge_df = pd.merge(ts_df, pdata_df, how='outer')
    # merge_df.to_csv('test.csv', index=False)
else:
    merge_df = ts_df

# away team lineup

In [130]:
atl = js.get('awayTeamLineUp')
abat = atl.get('batter')
apit = atl.get('pitcher')
hit_columns = ['name', 'pCode', 'posName', 'pos',
               'hitType', 'seqno', 'batOrder',
               'ab', 'hit', 'run', 'rbi',
               'hr', 'bb', 'so']
pit_columns = ['name', 'pCode', 'hitType', 'seqno',
               'inn', 'run', 'er', 'hit', 'hr',
               'bb', 'kk', 'hbp', 'wp', 'ballCount']
bats = pd.DataFrame(abat, columns=hit_columns).sort_values(['batOrder', 'seqno'])
pits = pd.DataFrame(apit, columns=pit_columns).sort_values('seqno')

In [129]:
pits

,name,pCode,hitType,seqno,inn,run,er,hit,hr,bb,kk,hbp,wp,ballCount
0,윌슨,68135,우투우타,1,6.0,2,2,7,0,4,2,1,0,96
1,고우석,67119,우투우타,2,1.2,0,0,1,0,0,1,0,0,28
2,진해수,76650,좌투좌타,3,0.1,0,0,0,0,0,0,0,0,4
3,김지용,60181,우투우타,4,1.0,1,1,1,1,0,1,0,0,21


In [131]:
bats

,name,pCode,posName,pos,hitType,seqno,batOrder,ab,hit,run,rbi,hr,bb,so
1,이형종,78135,중견수,8,우투우타,1,1,5,2,1,0,0,0,1
0,안익훈,65115,중견수,8,좌투좌타,2,1,1,0,0,0,0,0,0
2,오지환,79109,유격수,6,우투좌타,1,2,4,1,1,0,0,1,1
3,박용택,72133,지명타자,0,우투좌타,1,3,5,3,2,2,1,0,0
4,김현수,76290,좌익수,7,우투좌타,2,4,3,0,1,0,0,2,0
6,가르시아,68103,3루수,5,우투우타,1,5,5,2,1,1,0,0,0
5,윤진호,79198,3루수,5,우투우타,3,5,0,0,1,0,0,0,0
7,채은성,79192,우익수,9,우투우타,1,6,5,3,1,1,0,0,1
10,이천웅,61186,좌익수,7,좌투좌타,1,7,3,3,1,3,1,0,0
9,김용의,78217,1루수,3,우투좌타,2,7,0,0,0,0,0,0,0


In [45]:
bats.sort_values(['batOrder', 'seqno'])

,bb,ab,posName,seqno,batOrder,seasonHra,birth,weight,run,hr,...,pos,pCode,hbp,name,rbi,todayHra,so,height,cin,cout
0,0,3,중견수,1,1,0.4,19910304,85.0,0,0,...,8,64560,1,이창진,0,0.0,1,173.0,NaN,NaN
2,0,2,좌익수,2,2,0.3,19871226,87.0,1,0,...,7,76849,1,이명기,0,0.0,0,183.0,NaN,true
1,0,1,좌익수,4,2,0.2,19850519,105.0,0,0,...,7,78629,0,나지완,0,0.0,0,182.0,true,NaN
3,0,4,2루수,1,3,0.3,19900702,97.0,0,0,...,4,79608,0,안치홍,0,0.0,0,178.0,NaN,NaN
5,0,3,지명타자,1,4,0.2,19831216,106.0,1,0,...,0,72443,0,최형우,1,0.3,1,180.0,NaN,true
4,0,1,대타,2,4,0.3,19811125,96.0,0,0,...,10,70756,0,이범호,0,0.0,1,183.0,true,NaN
7,0,3,좌익수,1,5,0.2,19810325,94.0,0,0,...,7,70410,0,김주찬,0,0.0,1,183.0,NaN,true
6,0,1,우익수,2,5,0.1,19910726,75.0,0,0,...,9,64657,0,박준태,0,1.0,0,181.0,true,NaN
9,0,3,유격수,1,6,0.2,19891218,77.0,0,0,...,6,78603,0,김선빈,1,0.3,1,165.0,NaN,true
8,0,1,유격수,2,6,0.1,19930922,78.0,0,0,...,6,62908,0,황윤호,0,0.0,0,177.0,true,NaN


# download

- 지금은 모바일 페이지 들어가서 이것저것 긁어오는 중.
- `relayText.nhn`은 문자중계, 홈라인업, 어웨이라인업, 구장이름(`stadium`)
- `record.nhn`은 심판 이름(`referee`)
- `request`를 보낼 때 `params`를 파라미터로 같이 보내고, 이 중 `half` 파라미터에 이닝이 들어간다.
    - 이 이닝에 나온 문자중계만 `response`로 받음
- 리스폰스 객체의 `.json()` 함수로 `JSON` 형태의 데이터를 추출 가능
- 안에 `relayList`, `currentInning`, `homeTeamLineUp`, `awayTeamLineUp`, `schedule` 등의 키를 활용
    - `relayList`: 문자중계 내역
    - `currentInning`: 실시간 현재 이닝 -> 경기 끝난 후에는 경기 마지막 이닝
    - `homeTeamLineUp`: 홈팀 라인업
    - `awayTeamLineUp`: 어웨이 라인업
    - `schedule`: 진행중인 중계 관련 내용이 있는데 여기서 `stadium` 키를 쓰면 구장이 나온다

- `relayList`로 얻어온 value는 리스트 형태, 별도의 의미있는 인덱스는 없음(0, 1, ...), 리스트 엘리먼트 각각은 dictionary
- `relayList` 딕셔너리 내부에 `no`라는 키가 있는데 경기 문자중계 전체 단위 인덱스(순서)
    - 0번이 시작(__1회초 XX 공격__), 마지막 N번이 경기종료 메시지(__승리투수 등 표시__)
- 문자중계 내용은 `relayList` 딕셔너리의 `textOptionList` 키로 얻을 수 있는 리스트에 있음
- PTS 데이터는 `relayList` 딕셔너리의 `ptsOptionList` 키로 얻을 수 있는 리스트에 있음
    - 경우에 따라서 누락된 데이터도 있음
    - 순서가 꼭 문자중계와 맞지는 않음
    - ptsPitchId 키 값으로 매치해야함
- `textOptionList` 리스트 안의 엘리먼트 하나는 또 딕셔너리 object
    - 1구1구 단위 메타데이터 포함
    - 꼭 필요한(항상 나오는) 요소는...
        - `seqno`
        - `text`
        - `type`
        - `stuff` 등등
- 지금까지는 json으로 얻어온 object를 JSON 포맷으로 통채로 저장
- 앞으로는 object를 dataframe으로 바꾼 다음에 csv 형태로 보기 편하게 바꿔서 저장하는 쪽으로

- 다운로드 과정에서 Unicode 변환 안되는 텍스트는 누락 처리(blank text, '')했음
    - `try`, `catch` 써서 `UnicodeEncodeError` 나올 때 별도처리
- `.encode` 함수의 `errors` 파라미터를 `ignore`로 설정하면 에러 없이 해결 가능
    - 그렇게 바꿈

In [2]:
from pfx_download import *

In [3]:
args = [4, 4, 2019, 2019]

relay_url = 'http://m.sports.naver.com/ajax/baseball/gamecenter/kbo/relayText.nhn'
record_url = 'http://m.sports.naver.com/ajax/baseball/gamecenter/kbo/record.nhn'

game_ids = get_game_ids(args)

In [4]:
game_id = '20190423HTLG02019'

In [5]:
params = {
    'gameId': game_id,
    'half': '1'
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/59.0.3071.115 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'Host': 'm.sports.naver.com',
    'Referer': 'http://m.sports.naver.com/baseball/gamecenter/kbo/index.nhn?&gameId='
               + game_id
               + '&tab=relay'
}

response = requests.get(relay_url, params=params, headers=headers)

In [6]:
txt = {}
js = response.json()
if isinstance(js, str):
    js = json.loads(js)
last_inning = js['currentInning']

txt['relayList'] = {}

In [7]:
for i in range(len(js['relayList'])):
    txt['relayList'][js['relayList'][i]['no']] = js['relayList'][i]
txt['homeTeamLineUp'] = js['homeTeamLineUp']
txt['awayTeamLineUp'] = js['awayTeamLineUp']

txt['stadium'] = js['schedule']['stadium']

response.close()

In [20]:
for inn in range(2, last_inning + 1):
    params = {
        'gameId': game_id,
        'half': str(inn)
    }

    response = requests.get(relay_url, params=params, headers=headers)
    if response is not None:
        js = response.json()
        response.close()
        if isinstance(js, str):
            js = json.loads(js)

        # BUGBUG
        # 문자중계 텍스트에 비 unicode 문자가 들어간 경우.
        # gameid : 20180717LGWO02018
        # 문제가 되는 텍스트: \ufffd (REPLACEMENT CHARACTER) - cp949로 저장 불가
        # 해결책: cp949로 encoding 불가능한 문자가 있을 때는 blank text로 교체.
        for i in range(len(js['relayList'])):
            txt['relayList'][js['relayList'][i]['no']] = js['relayList'][i]
            texts = txt['relayList'][js['relayList'][i]['no']]['textOptionList']
            for i in range(len(texts)):
                try:
                    texts[i]['text'].encode('cp949', errors='ignore')
                except UnicodeEncodeError:
                    continue

    response.close()

In [176]:
params = {
    'gameId': game_id
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, '
                  'like Gecko) Chrome/59.0.3071.115 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'Host': 'm.sports.naver.com',
    'Referer': 'http://m.sports.naver.com/baseball/gamecenter/kbo/index.nhn?gameId='
               + game_id
               + '&tab=record'
}

response = requests.get(record_url, params=params, headers=headers)

p = regex.compile('(?<=\"etcRecords\":\[)[\\\.\{\}\"0-9:\s\(\)\,\ba-z가-힣\{\}]+')
result = p.findall(response.text)
if len(result) == 0:
    txt['referee'] = ''
else:
    txt['referee'] = result[0].split('{')[-1].split('":"')[1].split(' ')[0]

response.close()

# modify download

In [80]:
args = [4, 4, 2019, 2019]

relay_url = 'http://m.sports.naver.com/ajax/baseball/gamecenter/kbo/relayText.nhn'
record_url = 'http://m.sports.naver.com/ajax/baseball/gamecenter/kbo/record.nhn'

# game_ids = get_game_ids(args)
game_id = '20180717LGWO02018'

In [81]:
params = {
    'gameId': game_id,
    'half': '1'
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/59.0.3071.115 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'Host': 'm.sports.naver.com',
    'Referer': 'http://m.sports.naver.com/baseball/gamecenter/kbo/index.nhn?&gameId='
               + game_id
               + '&tab=relay'
}

response = requests.get(relay_url, params=params, headers=headers)

txt = {}
js = response.json()
if isinstance(js, str):
    js = json.loads(js)
last_inning = js['currentInning']

txt['relayList'] = {}

for i in range(len(js['relayList'])):
    txt['relayList'][js['relayList'][i]['no']] = js['relayList'][i]
txt['homeTeamLineUp'] = js['homeTeamLineUp']
txt['awayTeamLineUp'] = js['awayTeamLineUp']

txt['stadium'] = js['schedule']['stadium']

response.close()

In [83]:
for inn in range(2, last_inning + 1):
    params = {
        'gameId': game_id,
        'half': str(inn)
    }

    response = requests.get(relay_url, params=params, headers=headers)
    if response is not None:
        js = response.json()
        response.close()
        if isinstance(js, str):
            js = json.loads(js)

        for i in range(len(js['relayList'])):
            txt['relayList'][js['relayList'][i]['no']] = js['relayList'][i]
            texts = txt['relayList'][js['relayList'][i]['no']]['textOptionList']
            for i in range(len(texts)):
                texts[i]['text'].encode('cp949', 'ignore')
    response.close()

In [85]:
params = {
    'gameId': game_id
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, '
                  'like Gecko) Chrome/59.0.3071.115 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'Host': 'm.sports.naver.com',
    'Referer': 'http://m.sports.naver.com/baseball/gamecenter/kbo/index.nhn?gameId='
               + game_id
               + '&tab=record'
}

response = requests.get(record_url, params=params, headers=headers)

p = regex.compile('(?<=\"etcRecords\":\[)[\\\.\{\}\"0-9:\s\(\)\,\ba-z가-힣\{\}]+')
result = p.findall(response.text)
if len(result) == 0:
    txt['referee'] = ''
else:
    txt['referee'] = result[0].split('{')[-1].split('":"')[1].split(' ')[0]
    
response.close()

In [96]:
rl = txt['relayList']

tl_keys = []
rl_keys = []
pts_keys = []
for k in rl.keys():
    keys = rl.get(k).keys()
    for key in keys:
        if key in rl_keys:
            continue
        else:
            rl_keys.append(key)
    
    for j in range(len(rl.get(k).get('textOptionList'))):
        keys = rl.get(k).get('textOptionList')[j].keys()
        for key in keys:
            if key in tl_keys:
                continue
            else:
                tl_keys.append(key)
    for j in range(len(rl.get(k).get('ptsOptionList'))):
        keys = rl.get(k).get('ptsOptionList')[j].keys()
        for key in keys:
            if key in pts_keys:
                continue
            else:
                pts_keys.append(key)

tl_keys_copy = tl_keys.copy()
tl_keys_copy.remove('currentGameState')
tl_keys_copy.remove('batterRecord')
tl_keys_copy.remove('playerChange')
tl_keys_copy.remove('pitcherResult')

ts_set = []
for k in rl.keys():
    for j in range(len(rl.get(k).get('textOptionList'))):
        ts = rl.get(k).get('textOptionList')[j]
        
        ts_dict = {}
        ts_dict['textOrder'] = int(k)
        for key in tl_keys_copy:
            ts_dict[key] = None if key not in ts.keys() else ts.get(key)
        ts_dict['referee'] = referee
        ts_dict['stadium'] = stadium
        ts_set.append(ts_dict)
ts_df = pd.DataFrame(ts_set)
ts_df = ts_df.rename(index=str, columns={'ptsPitchId': 'pitchId'})

pdata_set = []
if len(pts_keys) > 0:
    for k in rl.keys():
        for j in range(len(rl.get(k).get('ptsOptionList'))):
            pdata = rl.get(k).get('ptsOptionList')[j]

            pdata_dict = {}
            pdata_dict['textOrder'] = int(k)
            for key in pts_keys:
                pdata_dict[key] = None if key not in pdata.keys() else pdata.get(key)
            pdata_set.append(pdata_dict)
        
    pdata_df = pd.DataFrame(pdata_set)
    pdata_df.head()
else:
    pdata_df = None

if pdata_df is not None:
    merge_df = pd.merge(ts_df, pdata_df, how='outer')
else:
    merge_df = ts_df

# team lineup

- 모바일 아닌 PC 페이지에서 가져와야 한다.
- 라인업 내용은 페이지 html에 하드코딩되어있지 않다.
- jquery? 사용한 자바스크립트 형태(아마도) 스크립트로 경기 메타를 object로 저장하고 있다.
- 이걸 불러와서 페이지에 뿌리는 식이다.
- 그래서 다음 순서로 진행한다.
    1. PC 페이지 html에서 BS4를 써서 `<script>` 있는 부분만 싹 긁어온다.
    2. 원하는 object 코드가 있는 스크립트를 찾는다. (object를 define하는 코드 텍스트를 검색)
    3. object 부분만 긁어온다. JSON 호환가능한 코드
    4. JSON 객체로 바꾼다.
    5. 라인업 부분만 가져온다.

- 가져온 라인업 내용은 batting order 순서대로 나열되어있다.
- key를 `'pos'`로 입력해서 포지션 내용만 가져올 수 있다.
- 맨앞의 글자가 선발 당시 라인업이다.
    - 도중 교체 출전은 '교', 지명타자는 '지'
    - 1, 2, 3루수는 한자로 一, 二, 三
    - 나머지는 포, 유, 좌, 중, 우

- 결과 기록 페이지라서 게임단위 메타 데이터만 있다.
- 선수 별로는 pCode 정도가 전부
- 모바일 페이지에서 긁어온 것과 합쳐야 한다.

In [25]:
lineup_url = 'https://sports.news.naver.com/gameCenter/gameRecord.nhn?category=kbo&gameId='
lurl = lineup_url + game_id
r = requests.get(lurl)
s = BeautifulSoup(r.text, 'lxml')
r.close()

In [26]:
a = s.find_all('script')
text = None

for tag in a:
    if len(tag.contents) > 0:
        if tag.contents[0].find('DataClass = ') > 0:
            text = tag.contents[0]
            start = text.find('DataClass = ') + 36
            end = text.find('}}}') + 3
            js = text[start:end]
            text = json.loads(js)
            break
            
bbs = text.get('battersBoxscore')
al = bbs.get('away')
hl = bbs.get('home')

In [29]:
aways = []

for i in range(len(al)):
    player = al[i]
    name = player.get('name')
    pos = player.get('pos')[0]
    aways.append({'name': name, 'pos': pos})
for elem in aways:
    print(elem)

{'name': '이형종', 'pos': '중'}
{'name': '안익훈', 'pos': '교'}
{'name': '오지환', 'pos': '유'}
{'name': '박용택', 'pos': '지'}
{'name': '김현수', 'pos': '一'}
{'name': '가르시아', 'pos': '三'}
{'name': '윤진호', 'pos': '교'}
{'name': '채은성', 'pos': '우'}
{'name': '이천웅', 'pos': '좌'}
{'name': '김용의', 'pos': '교'}
{'name': '양석환', 'pos': '교'}
{'name': '유강남', 'pos': '포'}
{'name': '정주현', 'pos': '二'}


In [30]:
homes = []

for i in range(len(hl)):
    player = hl[i]
    name = player.get('name')
    pos = player.get('pos')[0]
    homes.append({'name': name, 'pos': pos})
for elem in homes:
    print(elem)

{'name': '김혜성', 'pos': '二'}
{'name': '이택근', 'pos': '좌'}
{'name': '김하성', 'pos': '유'}
{'name': '초이스', 'pos': '우'}
{'name': '고종욱', 'pos': '지'}
{'name': '김민성', 'pos': '三'}
{'name': '김규민', 'pos': '一'}
{'name': '임병욱', 'pos': '중'}
{'name': '김재현', 'pos': '포'}
{'name': '장영석', 'pos': '교'}
{'name': '주효상', 'pos': '교'}


# 버그있던 경기 체크

In [31]:
lineup_url = 'https://sports.news.naver.com/gameCenter/gameRecord.nhn?category=kbo&gameId='
lurl = lineup_url + '20190423HTLG02019'
r = requests.get(lurl)
s = BeautifulSoup(r.text, 'lxml')
r.close()

In [32]:
a = s.find_all('script')
text = None

for tag in a:
    if len(tag.contents) > 0:
        if tag.contents[0].find('DataClass = ') > 0:
            text = tag.contents[0]
            start = text.find('DataClass = ') + 36
            end = text.find('}}}') + 3
            js = text[start:end]
            text = json.loads(js)
            break
            
bbs = text.get('battersBoxscore')
al = bbs.get('away')
hl = bbs.get('home')

In [34]:
aways = []

for i in range(len(al)):
    player = al[i]
    name = player.get('name')
    pos = player.get('pos')[0]
    aways.append({'name': name, 'pos': pos})
for elem in aways:
    print(elem)

{'name': '이창진', 'pos': '중'}
{'name': '이명기', 'pos': '우'}
{'name': '나지완', 'pos': '교'}
{'name': '안치홍', 'pos': '二'}
{'name': '최형우', 'pos': '지'}
{'name': '이범호', 'pos': '교'}
{'name': '김주찬', 'pos': '좌'}
{'name': '박준태', 'pos': '교'}
{'name': '김선빈', 'pos': '유'}
{'name': '황윤호', 'pos': '교'}
{'name': '류승현', 'pos': '一'}
{'name': '한승택', 'pos': '포'}
{'name': '김민식', 'pos': '교'}
{'name': '박찬호', 'pos': '三'}


In [35]:
homes = []

for i in range(len(hl)):
    player = hl[i]
    name = player.get('name')
    pos = player.get('pos')[0]
    homes.append({'name': name, 'pos': pos})
for elem in homes:
    print(elem)

{'name': '이천웅', 'pos': '중'}
{'name': '신민재', 'pos': '교'}
{'name': '오지환', 'pos': '유'}
{'name': '류형우', 'pos': '교'}
{'name': '김현수', 'pos': '좌'}
{'name': '채은성', 'pos': '우'}
{'name': '전민수', 'pos': '교'}
{'name': '유강남', 'pos': '포'}
{'name': '정상호', 'pos': '교'}
{'name': '박용택', 'pos': '지'}
{'name': '김민성', 'pos': '三'}
{'name': '윤진호', 'pos': '교'}
{'name': '김용의', 'pos': '一'}
{'name': '정주현', 'pos': '二'}


In [36]:
pos_dict = {'중': '중견수', '좌': '좌익수', '우': '우익수', '유': '유격수', '포': '포수', '지': '지명타자',
            '一': '1루수', '二': '2루수', '三': '3루수'}

posnum_dict = {'중': 8, '좌': 7, '우': 9, '유': 6, '포': 2, '지': 0,
            '一': 3, '二': 4, '三': 5}

In [46]:
bats2 = bats.copy()

In [47]:
for a in aways:
    if a.get('pos') == '교':
        continue
    bats2.loc[bats2.name == a.get('name'), 'posName'] = pos_dict.get(a.get('pos'))
    bats2.loc[bats2.name == a.get('name'), 'pos'] = posnum_dict.get(a.get('pos'))

In [48]:
bats2.sort_values(['batOrder', 'seqno'])

,bb,ab,posName,seqno,batOrder,seasonHra,birth,weight,run,hr,...,pos,pCode,hbp,name,rbi,todayHra,so,height,cin,cout
0,0,3,중견수,1,1,0.4,19910304,85.0,0,0,...,8,64560,1,이창진,0,0.0,1,173.0,NaN,NaN
2,0,2,우익수,2,2,0.3,19871226,87.0,1,0,...,9,76849,1,이명기,0,0.0,0,183.0,NaN,true
1,0,1,좌익수,4,2,0.2,19850519,105.0,0,0,...,7,78629,0,나지완,0,0.0,0,182.0,true,NaN
3,0,4,2루수,1,3,0.3,19900702,97.0,0,0,...,4,79608,0,안치홍,0,0.0,0,178.0,NaN,NaN
5,0,3,지명타자,1,4,0.2,19831216,106.0,1,0,...,0,72443,0,최형우,1,0.3,1,180.0,NaN,true
4,0,1,대타,2,4,0.3,19811125,96.0,0,0,...,10,70756,0,이범호,0,0.0,1,183.0,true,NaN
7,0,3,좌익수,1,5,0.2,19810325,94.0,0,0,...,7,70410,0,김주찬,0,0.0,1,183.0,NaN,true
6,0,1,우익수,2,5,0.1,19910726,75.0,0,0,...,9,64657,0,박준태,0,1.0,0,181.0,true,NaN
9,0,3,유격수,1,6,0.2,19891218,77.0,0,0,...,6,78603,0,김선빈,1,0.3,1,165.0,NaN,true
8,0,1,유격수,2,6,0.1,19930922,78.0,0,0,...,6,62908,0,황윤호,0,0.0,0,177.0,true,NaN


In [50]:
bats.sort_values(['batOrder', 'seqno'])

,bb,ab,posName,seqno,batOrder,seasonHra,birth,weight,run,hr,...,pos,pCode,hbp,name,rbi,todayHra,so,height,cin,cout
0,0,3,중견수,1,1,0.4,19910304,85.0,0,0,...,8,64560,1,이창진,0,0.0,1,173.0,NaN,NaN
2,0,2,좌익수,2,2,0.3,19871226,87.0,1,0,...,7,76849,1,이명기,0,0.0,0,183.0,NaN,true
1,0,1,좌익수,4,2,0.2,19850519,105.0,0,0,...,7,78629,0,나지완,0,0.0,0,182.0,true,NaN
3,0,4,2루수,1,3,0.3,19900702,97.0,0,0,...,4,79608,0,안치홍,0,0.0,0,178.0,NaN,NaN
5,0,3,지명타자,1,4,0.2,19831216,106.0,1,0,...,0,72443,0,최형우,1,0.3,1,180.0,NaN,true
4,0,1,대타,2,4,0.3,19811125,96.0,0,0,...,10,70756,0,이범호,0,0.0,1,183.0,true,NaN
7,0,3,좌익수,1,5,0.2,19810325,94.0,0,0,...,7,70410,0,김주찬,0,0.0,1,183.0,NaN,true
6,0,1,우익수,2,5,0.1,19910726,75.0,0,0,...,9,64657,0,박준태,0,1.0,0,181.0,true,NaN
9,0,3,유격수,1,6,0.2,19891218,77.0,0,0,...,6,78603,0,김선빈,1,0.3,1,165.0,NaN,true
8,0,1,유격수,2,6,0.1,19930922,78.0,0,0,...,6,62908,0,황윤호,0,0.0,0,177.0,true,NaN


# pbp_download(new)

In [54]:
def download_relay(args, lm=None):
    # return True or False
    relay_url = 'http://m.sports.naver.com/ajax/baseball/gamecenter/kbo/relayText.nhn'
    record_url = 'http://m.sports.naver.com/ajax/baseball/gamecenter/kbo/record.nhn'

    now = datetime.datetime.now()
    today_year = now.year
    today_date = int(now.date().strftime('%m%d'))
    
    game_ids = get_game_ids(args)
    if (game_ids is None) or (len(game_ids) == 0):
        print('no game ids')
        print('args: {}'.format(args))
        if lm is not None:
            lm.log('no game ids')
            lm.log('args: {}'.format(args))
        return False

    if lm is not None:
        lm.resetLogHandler()
        lm.setLogPath(os.getcwd())
        lm.setLogFileName('relay_download_log.txt')
        lm.cleanLog()
        lm.createLogHandler()
        lm.log('---- Relay Text Download Log ----')

    if not os.path.isdir('pbp_data'):
        os.mkdir('pbp_data')
    os.chdir('pbp_data')
    # path: pbp_data

    print("##################################################")
    print("######        DOWNLOAD RELAY DATA          #######")
    print("##################################################")

    for year in game_ids.keys():
        start1 = time.time()
        print(" Year {}".format(year))
        if len(game_ids[year]) == 0:
            print('month id is empty')
            print('args: {}'.format(args))
            if lm is not None:
                lm.log('month id is empty')
                lm.log('args : {}'.format(args))
            return False

        if not os.path.isdir(str(year)):
            os.mkdir(str(year))
        os.chdir(str(year))
        # path: pbp_data/year

        for month in game_ids[year].keys():
            start2 = time.time()
            print("  Month {}".format(month))
            if len(game_ids[year][month]) == 0:
                print('month id is empty')
                print('args: {}'.format(args))
                if lm is not None:
                    lm.log('month id is empty')
                    lm.log('args : {}'.format(args))
                return False

            if not os.path.isdir(str(month)):
                os.mkdir(str(month))
            os.chdir(str(month))
            # path: pbp_data/year/month

            # download
            done = 0
            skipped = 0
            for game_id in game_ids[year][month]:
                game_id_year = int(game_id[:4])
                game_id_date = int(game_id[4:8])
                game_id_team = game_id[8:10]
                if (game_id_year < 2008) or (game_id_year > 7777):
                    skipped += 1
                    continue
                if (game_id_year == today_year) and (game_id_date > today_date):
                    skipped += 1
                    continue
                if game_id_date < int(regular_start[game_id[:4]]):
                    skipped += 1
                    continue
                if game_id_date >= int(playoff_start[game_id[:4]]):
                    skipped += 1
                    continue
                if game_id_team not in teams:
                    skipped += 1
                    continue

                if not check_url(relay_url):
                    skipped += 1
                    if lm is not None:
                        lm.log('URL error : {}'.format(relay_url))
                    continue

                relay_output_name = game_id + '_relay.csv'
                if (int(game_id[:4]) == today_year) &\
                   (int(game_id[4:6]) == now.month) &\
                   (int(game_id[6:8]) == now.day):
                       done = done
                elif (os.path.isfile(relay_output_name)) and \
                        (os.path.getsize(relay_output_name) > 0):
                    done += 1
                    if lm is not None:
                        lm.log('File Duplicate : {}'.format(game_id))
                    continue

                params = {
                    'gameId': game_id,
                    'half': '1'
                }

                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
                                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                                  'Chrome/59.0.3071.115 Safari/537.36',
                    'X-Requested-With': 'XMLHttpRequest',
                    'Host': 'm.sports.naver.com',
                    'Referer': 'http://m.sports.naver.com/baseball/gamecenter/kbo/index.nhn?&gameId='
                               + game_id
                               + '&tab=relay'
                }

                response = requests.get(relay_url, params=params, headers=headers)

                if response is not None:
                    txt = {}
                    js = response.json()
                    if isinstance(js, str):
                        js = json.loads(js)
                    last_inning = js['currentInning']

                    if last_inning is None:
                        skipped += 1
                        lm.log('Gameday not found : {}'.format(game_id))
                        continue

                    txt['relayList'] = {}
                    for i in range(len(js['relayList'])):
                        text_index = js['relayList'][i]['no']
                        txt['relayList'][text_index] = js['relayList'][i]
                        texts = txt['relayList'][text_index]['textOptionList']
                        for i in range(len(texts)):
                            texts[i]['text'].encode('cp949', 'ignore')
                    txt['homeTeamLineUp'] = js['homeTeamLineUp']
                    txt['awayTeamLineUp'] = js['awayTeamLineUp']

                    txt['stadium'] = js['schedule']['stadium']

                    response.close()

                    for inn in range(2, last_inning + 1):
                        params = {
                            'gameId': game_id,
                            'half': str(inn)
                        }

                        response = requests.get(relay_url, params=params, headers=headers)
                        if response is not None:
                            js = response.json()
                            if isinstance(js, str):
                                js = json.loads(js)
                            for i in range(len(js['relayList'])):
                                txt['relayList'][js['relayList'][i]['no']] = js['relayList'][i]
                                texts = txt['relayList'][js['relayList'][i]['no']]['textOptionList']
                                for i in range(len(texts)):
                                    texts[i]['text'].encode('cp949', 'ignore')
                        else:
                            skipped += 1
                            if lm is not None:
                                lm.log('Cannot get response : {}'.format(game_id))

                        response.close()

                    # get referee
                    params = {
                        'gameId': game_id
                    }

                    headers = {
                        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, '
                                      'like Gecko) Chrome/59.0.3071.115 Safari/537.36',
                        'X-Requested-With': 'XMLHttpRequest',
                        'Host': 'm.sports.naver.com',
                        'Referer': 'http://m.sports.naver.com/baseball/gamecenter/kbo/index.nhn?gameId='
                                   + game_id
                                   + '&tab=record'
                    }

                    response = requests.get(record_url, params=params, headers=headers)

                    p = regex.compile('(?<=\"etcRecords\":\[)[\\\.\{\}\"0-9:\s\(\)\,\ba-z가-힣\{\}]+')
                    result = p.findall(response.text)
                    if len(result) == 0:
                        txt['referee'] = ''
                    else:
                        txt['referee'] = result[0].split('{')[-1].split('":"')[1].split(' ')[0]

                    response.close()
                    
                    
                    ### 필요한 내용 담아서 저장 ###
                    rl = txt['relayList']

                    tl_keys = []
                    rl_keys = []
                    pts_keys = []
                    for k in rl.keys():
                        keys = rl.get(k).keys()
                        for key in keys:
                            if key in rl_keys:
                                continue
                            else:
                                rl_keys.append(key)

                        for j in range(len(rl.get(k).get('textOptionList'))):
                            keys = rl.get(k).get('textOptionList')[j].keys()
                            for key in keys:
                                if key in tl_keys:
                                    continue
                                else:
                                    tl_keys.append(key)
                        for j in range(len(rl.get(k).get('ptsOptionList'))):
                            keys = rl.get(k).get('ptsOptionList')[j].keys()
                            for key in keys:
                                if key in pts_keys:
                                    continue
                                else:
                                    pts_keys.append(key)

                    tl_keys_copy = tl_keys.copy()
                    tl_keys_copy.remove('currentGameState')
                    tl_keys_copy.remove('batterRecord')
                    tl_keys_copy.remove('playerChange')
                    tl_keys_copy.remove('pitcherResult')

                    ts_set = []
                    for k in rl.keys():
                        for j in range(len(rl.get(k).get('textOptionList'))):
                            ts = rl.get(k).get('textOptionList')[j]

                            ts_dict = {}
                            ts_dict['textOrder'] = int(k)
                            for key in tl_keys_copy:
                                ts_dict[key] = None if key not in ts.keys() else ts.get(key)
                            ts_dict['referee'] = referee
                            ts_dict['stadium'] = stadium
                            ts_set.append(ts_dict)
                    ts_df = pd.DataFrame(ts_set)
                    ts_df = ts_df.rename(index=str, columns={'ptsPitchId': 'pitchId'})

                    pdata_set = []
                    if len(pts_keys) > 0:
                        for k in rl.keys():
                            for j in range(len(rl.get(k).get('ptsOptionList'))):
                                pdata = rl.get(k).get('ptsOptionList')[j]

                                pdata_dict = {}
                                pdata_dict['textOrder'] = int(k)
                                for key in pts_keys:
                                    pdata_dict[key] = None if key not in pdata.keys() else pdata.get(key)
                                pdata_set.append(pdata_dict)

                        pdata_df = pd.DataFrame(pdata_set)
                        pdata_df.head()
                    else:
                        pdata_df = None

                    if pdata_df is not None:
                        merge_df = pd.merge(ts_df, pdata_df, how='outer')
                    else:
                        merge_df = ts_df
                    ### 라인업 가져다와서 더하기 ###

                    fp = open(game_id + '_relay.json', 'w', newline='\n')
                    json.dump(txt, fp, ensure_ascii=False, sort_keys=False, indent=4)
                    fp.close()
                    
                    ##### 텍스트만 저장
                    text_list = []
                    pts_list = []
                    text_list_header = ["textOrder","textType","text","ptsPitchId","stuff","speed"]
                    pts_list_header = ["textOrder","inn","ballcount","crossPlateX",
                                       "topSz","crossPlateY","pitchId","vy0","vz0",
                                       "vx0","z0","y0","ax","x0","ay","az","bottomSz","stance"]
                    for k in sorted(txt['relayList'].keys()):
                        textset = txt['relayList'][k]
                        textOptionList = textset['textOptionList']
                        for to in textOptionList:
                            row = [k, to['type'], to['text']]
                            if 'ptsPitchId' in to.keys():
                                row.append(to['ptsPitchId'])
                            else:
                                row.append('')
                            if 'stuff' in to.keys():
                                row.append(to['stuff'])
                            else:
                                row.append('')
                            if 'speed' in to.keys():
                                row.append(to['speed'])
                            else:
                                row.append('')
                            text_list.append(row)
                        if 'ptsOptionList' in textset.keys():
                            ptsOptionList = textset['ptsOptionList']
                            for po in ptsOptionList:
                                row = [k] + list(po.values())
                                pts_list.append(row)

                    fp = open(game_id + '_textset.csv', 'w', newline='\n')
                    cf = csv.writer(fp)
                    cf.writerow(text_list_header)
                    for tl in text_list:
                        cf.writerow(tl)
                    fp.close()

                    fp = open(game_id + '_ptsset.csv', 'w', newline='\n')
                    cf = csv.writer(fp)
                    cf.writerow(pts_list_header)
                    for pl in pts_list:
                        cf.writerow(pl)
                    fp.close()
                    #####

                    done += 1
                else:
                    skipped += 1
                    if lm is not None:
                        lm.log('Cannot get response : {}'.format(game_id))

                print_progress('    Downloading: ', len(game_ids[year][month]), done, skipped)

            # download done
            print_progress('    Downloading: ', len(game_ids[year][month]), done, skipped)
            print('\n        Downloaded {} files'.format(done))
            print('        (Skipped {} files)'.format(skipped))
            end2 = time.time()
            print('            -- elapsed {:.3f} sec for month {}'.format(end2 - start2, month))

            os.chdir('..')
            # path: pbp_data/year
        end1 = time.time()
        print('   -- elapsed {:.3f} sec for year {}'.format(end1 - start1, year))
        # months done
        os.chdir('..')
        # path: pbp_data/
    # years done
    os.chdir('..')
    # path: root
    return True